In [ ]:
import gc
import chardet
import re
import os
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64
print(os.listdir("../input"))

In [ ]:
from keras.models import Model, Sequential
from keras.layers import Dense, Embedding, Input, Conv1D, GlobalMaxPool1D, Dropout, concatenate, Layer, InputSpec, CuDNNLSTM, Bidirectional
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from keras import activations, initializers, regularizers, constraints
from keras.utils.conv_utils import conv_output_length
from keras.regularizers import l2
from keras.constraints import maxnorm

In [ ]:
def f1_score(true,pred): #considering sigmoid activation, threshold = 0.5
    pred = K.cast(K.greater(pred,0.5), K.floatx())

    groundPositives = K.sum(true) + K.epsilon()
    correctPositives = K.sum(true * pred) + K.epsilon()
    predictedPositives = K.sum(pred) + K.epsilon()

    precision = correctPositives / predictedPositives
    recall = correctPositives / groundPositives

    m = (2 * precision * recall) / (precision + recall)

    return m

In [ ]:
train_df = pd.read_csv('../input/train.csv', usecols=['question_text', 'target'])
test_df = pd.read_csv('../input/test.csv', usecols = ['question_text'])

In [ ]:
train_df['question_text'] = train_df['question_text'].apply(lambda x: x.lower())
test_df['question_text'] = test_df['question_text'].apply(lambda x: x.lower())

pattern = re.compile(r"[^a-z ]+")
train_df['question_text'] = train_df['question_text'].apply(lambda x: re.sub(pattern, "", x))
test_df['question_text'] = test_df['question_text'].apply(lambda x: re.sub(pattern, "", x))

In [ ]:
train_sentences = list(train_df['question_text'])
train_labels = np.array(list(train_df['target']))
test_sentences = list(test_df['question_text'])

In [ ]:
gc.collect()

In [ ]:
max_features = 20000
maxlen = 100

In [ ]:
tokenizer = text.Tokenizer(num_words=max_features)

In [ ]:
tokenizer.fit_on_texts(train_sentences)

In [ ]:
tokenized_train = tokenizer.texts_to_sequences(train_sentences)
X_train = sequence.pad_sequences(tokenized_train, maxlen=maxlen)

In [ ]:
X_train[0]

In [ ]:
tokenized_test = tokenizer.texts_to_sequences(test_sentences)
X_test = sequence.pad_sequences(tokenized_test, maxlen=maxlen)

In [ ]:
del tokenized_test, tokenized_train, tokenizer, train_sentences, test_sentences
gc.collect()

In [ ]:
batch_size = 1024
epochs = 5
embed_size = 300

In [ ]:
gc.collect()

In [ ]:
def cudnnlstm_model():
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size)(inp)
    x_1 = Bidirectional(CuDNNLSTM(256, kernel_initializer='glorot_normal', 
                                kernel_regularizer=l2(4e-6), bias_regularizer=l2(4e-6), kernel_constraint=maxnorm(10), bias_constraint=maxnorm(10)))(x)
    x_2 = Bidirectional(CuDNNLSTM(256, kernel_initializer='glorot_normal', 
                                kernel_regularizer=l2(4e-6), bias_regularizer=l2(4e-6), kernel_constraint=maxnorm(10), bias_constraint=maxnorm(10)))(x)
    x = concatenate([x_1, x_2])
    
    x = Dense(64, activation="tanh", kernel_initializer='glorot_normal')(x)
    x = Dense(1, activation="sigmoid", kernel_initializer='glorot_normal')(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=[f1_score])

    return model

In [ ]:
cudnnlstm_model = cudnnlstm_model()
cudnnlstm_model.summary()

In [ ]:
weight_path="early_weights.hdf5"
checkpoint = ModelCheckpoint(weight_path, monitor='val_f1_score', verbose=1, save_best_only=True, mode='max')
early_stopping = EarlyStopping(monitor="val_f1_score", mode="max", patience=5)
callbacks = [checkpoint, early_stopping]

In [ ]:
cudnnlstm_model.fit(X_train, train_labels, batch_size=batch_size, epochs=epochs, shuffle = True, validation_split=0.20, callbacks=callbacks)

In [ ]:
cudnnlstm_model.load_weights(weight_path)
y_pred = cudnnlstm_model.predict(X_test, batch_size=batch_size)
y_pred = [x for i in y_pred for x in i]

In [ ]:
sample = pd.read_csv('../input/sample_submission.csv')

In [ ]:
sample['prediction'] = pd.Series(y_pred)

In [ ]:
sample['prediction'] = sample['prediction'].apply(lambda x: 0 if x <= 0.5 else 1)

In [ ]:
sample.to_csv('submission.csv', index=False)